In [ ]:
import sys
import os
from matplotlib import pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
import gc

from augmentation import augment
from data_generator import DOTASequence, SegmentationSequence
import model_builder
import accuracy

In [ ]:
annot_path = 'data/train/annotations' #Path to directory of annotations
images_path = 'data/train/images' #Path to directory of images
generator = DOTASequence(images_path, annot_path, augmenter = augment)

In [ ]:
idx = 4
imgbatch, annotbatch = generator[idx]
img = imgbatch[0]
annot = annotbatch[0]

plt.figure(figsize = (25,25))

for line in annot:
    p1 = int(line[0]), int(line[1])
    p2 = int(line[2]), int(line[3])
    p3 = int(line[4]), int(line[5])
    p4 = int(line[6]), int(line[7])
    
    color = (0,255,255)
    cv2.line(img, p1, p2, color,2)
    cv2.line(img, p2, p3, color,2)
    cv2.line(img, p3, p4, color,2)
    cv2.line(img, p4, p1, color,2)

plt.imshow(img)

In [ ]:
# Dank new visualizer for mask utilities and revamp augmentation function below vvvvvvvv

import augmentation
import masks
import cv2
import os
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
img_path = "./data/train/images"
box_path = "./data/train/annotations"
mask_path = "./data/masks"

if not(os.path.exists(mask_path)):
    os.makedirs(mask_path)

image_names = [x[:-4] for x in os.listdir(img_path) if x.endswith(".png")]

In [ ]:
n = 1
index = image_names[n]

masks.save_mask(img_path, box_path, index, mask_path)
img = cv2.imread(os.path.join(img_path, index + '.png'))
mask = masks.load_mask(mask_path, index)

aimg, amask = augmentation.augment(img,mask)

agmask = np.zeros((amask.shape[0],amask.shape[1]),dtype = np.uint8)
for i in range(16):
    agmask = agmask + (amask[:,:,i])

plt.figure(figsize = (25,25))
plt.subplot(121)
plt.imshow(aimg)
plt.subplot(122)
plt.imshow(agmask)

In [ ]:
seg_generator = SegmentationSequence(images_path, annot_path, augmenter = augment, output_img_size=(5000, 3500))
img, mask = seg_generator[0]
img = img[0].astype(np.uint8)
mask = mask[0]
fig = plt.figure(figsize=(10,20))
fig.add_subplot(2, 1, 1)
plt.imshow(img)
fig.add_subplot(2, 1, 2)
plt.imshow(mask[:,:,0], cmap="hot", interpolation="nearest")

In [ ]:
model = model_builder.build_segmentation_model((None, None, 3), 4, 3, 8, 16)
imgbatch, annotbatch = generator[0]
fig = plt.figure(figsize=(25, 25))
for i, img in enumerate(imgbatch):
    padded_img = np.zeros(
        (1, (img.shape[0] // 16 + 1) * 16, (img.shape[1] // 16 + 1) * 16, 3)
    )
    padded_img[0, : img.shape[0], : img.shape[1], :] = img
    pred = model.predict(padded_img)
    pred = np.argmax(pred, axis=3)[0]
    fig.add_subplot(2, 5, i + 1)
    plt.imshow(padded_img[0].astype(np.uint8))
    fig.add_subplot(2, 5, i + 6)
    plt.imshow(pred, cmap="hot", interpolation="nearest")

In [ ]:
model = tf.keras.models.load_model(".\\ckpts", compile=False, custom_objects={'iou_coef': accuracy.iou_coef})

In [ ]:
seg_generator = SegmentationSequence(
        ".\\data\\train\\images", ".\\data\\train\\annotations", batch_size=1
    )
for i in range(30):
    img, mask = seg_generator[0]

img.shape = (1, 480, 480, 3)
mask.shape = (1, 480, 480, 16)
plt.figure(figsize=(10,10))
plt.imshow(img[0])
pred = model.predict(img)[0]
pred = pred * 255
pred = pred.astype(np.uint8)
fig = plt.figure(figsize=(8,48))
for cnum in range(16):
    fig.add_subplot(16, 2, 2 * cnum + 1)
    plt.imshow(mask[0,:,:,cnum], cmap='hot', interpolation="nearest")
    fig.add_subplot(16, 2, 2 * cnum + 2)
    plt.imshow(pred[:,:,cnum], cmap='hot', interpolation="nearest")

pred_max = np.argmax(pred, axis=2)
fig = plt.figure(figsize=(10,10))
plt.imshow(pred_max, cmap='hot', interpolation="nearest")

In [ ]:
tf.keras.backend.clear_session()
del model
gc.collect()